In [32]:
"""
compute_technical.py

Computes technical indicators from price_data and writes the results to a new PostgreSQL table.
"""

import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

# Setup DB connection
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# Load price data
query = """
SELECT ticker, date, close, volume
FROM price_data
ORDER BY ticker, date ASC;
"""
df = pd.read_sql(query, engine, parse_dates=["date"])

# Compute technical indicators per ticker
def compute_indicators(group):
    group = group.sort_values("date").copy()
    group["return_1d"] = group["close"].pct_change()
    group["sma_5"] = group["close"].rolling(window=5).mean()
    group["sma_20"] = group["close"].rolling(window=20).mean()
    group["ema_10"] = group["close"].ewm(span=10, adjust=False).mean()
    group["rsi_14"] = compute_rsi(group["close"], 14)
    group["macd"] = group["close"].ewm(span=12, adjust=False).mean() - group["close"].ewm(span=26, adjust=False).mean()
    return group

# RSI function
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Clean and apply without warning or dropping 'ticker'
features_df = (
    df.groupby("ticker", group_keys=False)
    .apply(compute_indicators)
    .reset_index(drop=True)
)

# Drop duplicates before inserting
unique_keys = features_df[['ticker', 'date']].drop_duplicates()

for i in range(len(unique_keys)):
    ticker = unique_keys.iloc[i]['ticker']
    date_val = pd.to_datetime(unique_keys.iloc[i]['date']).date()

    session.execute(
        text("DELETE FROM technical_features WHERE ticker = :ticker AND date = :date"),
        {"ticker": ticker, "date": date_val}
    )
session.commit()

# Append new data
features_df.to_sql('technical_features', engine, if_exists='append', index=False)
print("✅ Technical features deduplicated and saved to 'technical_features' table in PostgreSQL")


/var/folders/3y/jcf5sjb1319blyc1b_cfn1xm0000gn/T/ipykernel_66624/3128870013.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_indicators)


✅ Technical features deduplicated and saved to 'technical_features' table in PostgreSQL
